# 변수 분포 문제란?
- 일반화된 모델을 학습하는데 어려움이 있는 분포를 가지는 변수가 있어 학습하지 못하는 문제
- 이상치 포함, 특징 간 상관성, 변수 간 스케일 차이, 일반 분포, 특징과 라벨 간 약한 관계 및 비선형 관계

## 특징과 라벨 간 약한 관계 및 비선형 관계
- 특징과 라벨 간 관계가 없거나 매우 약하다면, 어떠한 전처리 및 모델링을 하더라도 예측력이 높은 모델을 학습할 수 없음
- 특징과 라벨 간 비선형 관계가 존재한다면, 적절한 전처리를 통해 모델 성능을 향상 가능
### 해결 방안
- 다양한 변환 방법을 사용하여 특징을 생성한 뒤 특징 선택을 수행해야 함

In [2]:
import pandas as pd
df = pd.read_csv('./데이터/Combined_Cycle_Power_Plant.csv')
x = df.drop('EP', axis = 1)
y = df['EP']

In [4]:
# 성능 비교를 위해 신규 데이터 생성
x_added = x.copy() # 특징이 추가된 데이터를 부착할 데이터

import numpy as np
# 로그와 제곱 관련 변수만 추가
for col in x.columns:
    x_added[col + '_squared'] = x[col] ** 2
    x_added[col + '_log'] = np.log(x[col])

In [5]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression as LR

# 5겹 교차 검증 기반의 평가 수행
x_score = cross_val_score(LR(), x, y, cv = 5, scoring = 'neg_mean_absolute_error').mean()
x_added_score = cross_val_score(LR(), x_added, y, cv = 5, scoring = 'neg_mean_absolute_error').mean()

In [6]:
print('특징 추가 전 : {}, 특징 추가 후 : {}'.format(x_score, x_added_score))

특징 추가 전 : -3.6282513807290435, 특징 추가 후 : -3.332221506783882


## 이상치 제거
- 이상치를 포함하는 레코드를 제거(절대 추정의 대상이 아님)
- IQR 규칙 활용
    - 직관적이고 사용이 간편
    - 단일 변수론 어려움
    - numpy.quantile

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
df = pd.read_csv('./데이터/glass.csv')
x = df.drop(['Glass_type'], axis = 1)
y = df['Glass_type']

train_x, test_x, train_y, test_y = train_test_split(x,y)
train_x.shape

(160, 8)

In [10]:
import numpy as np
def IQR_rule(val_list): # 한 특징에 포함된 값 [열 벡터]
    Q1 = np.quantile(val_list, 0.25)
    Q3 = np.quantile(val_list, 0.75)
    IQR = Q3 - Q1
    
    # IQR rule을 위배하지 않는 bool list 계산
    not_outlier_condition = (Q3 + 1.5 * IQR > val_list) & (Q1 - 1.5 * IQR < val_list)
    return not_outlier_condition

In [11]:
conditions = train_x.apply(IQR_rule) # apply를 이용해 모든 칼럼에 함수
conditions

,RI,Na,Mg,Al,Si,K,Ca,Fe
114,True,True,True,True,True,True,True,True
167,True,True,True,True,True,True,True,True
150,True,True,True,True,True,True,True,True
35,True,True,True,True,True,True,True,True
143,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...
148,True,True,True,True,True,True,True,True
147,True,True,True,True,True,True,True,True
32,True,True,True,True,True,True,True,True
175,True,True,True,True,True,True,True,True


In [13]:
total_condition = conditions.sum(axis = 1) == len(train_x.columns)
# 하나라도 IQR 규칙을 위반하는 요소를 갖는 레코드를 제거하기 위한 규칙
train_x = train_x[total_condition] # 이상치 제거

In [14]:
train_x.shape # 약 58개 삭제

(102, 8)

In [15]:
58/160 # 35%는 너무 커서 1.5 값을 조정해야함

0.3625

- 밀도 기반 군집화 수행
    - 군집에 속하지 않은 샘플을 이상치라고 간주
    - 파라미터 튜닝이 쉽지 않음
    - eps : 이웃이라 판단하는 반경
    - min_samples : eps내에 들어와야하는 최소 샘플 수
    - metric : 사용하는 거리 척도
    - .labels_ : 각 샘플이 속한 군집 정보(-1 : 이상치)

In [17]:
df = pd.read_csv('./데이터/glass.csv')
x = df.drop(['Glass_type'], axis = 1)
y = df['Glass_type']

train_x, test_x, train_y, test_y = train_test_split(x,y)
train_x.shape

(160, 8)

In [18]:
from sklearn.cluster import DBSCAN
from scipy.spatial.distance import cdist
DM = cdist(train_x, train_x) # 거리행렬 -> DBSCAN의 파라미터를 설정
np.quantile(DM, 0.1) # 샘플 간 거리의 10% quantile이 0.6738 정도

0.6738254889313998

In [19]:
cluster_model = DBSCAN(eps = 0.67, min_samples = 3).fit(train_x)
print(sum(cluster_model.labels_ == -1))
# 36개가 이상치로 판단 -> 이정도 너무 많음 -> 파라미터 조정

36


In [20]:
cluster_model = DBSCAN(eps = 2, min_samples = 3).fit(train_x)
print(sum(cluster_model.labels_ == -1))
# 8개 정도면 괜찮은 양이라고 판단

8


In [21]:
train_x = train_x[cluster_model.labels_ != -1]

In [22]:
train_x.shape

(152, 8)